### We first retrieve the full dataset. Then we sample this dataset (10% and 20% ) and tokenize samples into sentences (rule-based fast tokenization) with tokenize_uk.

In [5]:
import pandas as pd
import json
import fasttext
import emoji
import re
import collections
from urllib.parse import urlparse

In [7]:
df_posts = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/data_preselected_for_1st_annotation_proj/only_text.csv', header=0)

In [8]:
df_posts

,id,date,message,type,channel_name,lang
0,12347.0,2022-12-15 16:32:15+00:00,А трансляция тем временем идет. Подписывайтесь...,photo,mardanaka,ru
1,12346.0,2022-12-15 15:00:03+00:00,"В России ещё есть те, кто способен возрождать ...",photo,mardanaka,ru
2,12345.0,2022-12-15 14:21:22+00:00,«Катаргейт» набирает обороты - Обвиняемая в ко...,text,mardanaka,ru
3,12344.0,2022-12-15 13:08:35+00:00,🔥В гостях у @Metametrica побывали товарищи из ...,photo,mardanaka,ru
4,12343.0,2022-12-15 12:31:23+00:00,Глава Минобороны Грузии назвал грузинских наем...,text,mardanaka,ru
...,...,...,...,...,...,...
6836539,8.0,2018-10-18 14:18:13+00:00,Мария Топилина – муж – Максим Топилин – минист...,text,bloodysx,ru
6836540,7.0,2018-10-18 14:17:44+00:00,"Людмила Красноштанова – муж – член «ЕдРа», деп...",text,bloodysx,ru
6836541,6.0,2018-10-18 14:16:00+00:00,"И, на всякий случай, если вы все проспали, дуб...",text,bloodysx,ru
6836542,5.0,2018-10-18 14:00:26+00:00,"Вот вам доказательство, что это реальная я.",video,bloodysx,ru


In [9]:
df_sample = df_posts.sample(frac=0.2, random_state=42).copy()
df_sample.reset_index(inplace=True)

In [10]:
df_small_sample = df_posts.sample(frac=0.1, random_state=42).copy()
df_small_sample.reset_index(inplace=True)

In [11]:
import tokenize_uk

In [12]:
len(df_sample)

1367309

In [13]:
%%time

df_sample['tokenized_sents'] = df_sample['message'].apply(lambda x: tokenize_uk.tokenize_sents(x))

CPU times: user 36.3 s, sys: 15.9 s, total: 52.2 s
Wall time: 59.6 s


In [14]:
df_sample['tokenized_sents'][2000]

['#АлтайАлтаюРознь2 Не утихают федеральные телеграм-скандалы с упоминанием губернатора Алтайского края Карлина.',
 'В соседней Республике Алтай ситуация более стабильна.',
 'Глава региона вплотную занялся проблемами обманутых дольщиков и накануне провел встречу с пострадавшими от мошенников гражданами.',
 'После их обращения генерал Бердников изыскал средства в республиканском бюджете (46 млн) и на встрече с жителями заверил, что дом по улице Чаптынова, 34 в Горно-Алтайске на 60 квартир будет сдан уже в следующем году.',
 'Кстати, директор строительной фирмы «ГАСК» Борщев уже отбывает наказание в виде 6 лет лишения свободы.   http://www.interfax-russia.ru/Siberia/news.asp?id=911826&sec=1671']

## We set aside 1000 neutral sentences (not expected to contain neither dehumanization, not even mentions of Ukrainians)

In [15]:
df_sample['random_sents'] = df_sample.tokenized_sents.str[0]

In [16]:
neutral_sent_list = df_sample['random_sents'].sample(1000, random_state=42).tolist()

In [18]:
df_neutral = pd.DataFrame({'text': neutral_sent_list[:]})
df_neutral.to_csv('df_neutral.csv')

## Selecting the sentences that contain mentions of Ukraine, ukrainians + filtering by the tone

At the first iteration we track the sequences the define the use of the most common terms.

In [21]:
seqs_to_track = ['укрa', 'хох', 'хохл']

In [22]:
def find_sentences_w_sec(list_sent, seqs_to_track):
    sent_list_result = []
    for sent in list_sent:
        for seq in seqs_to_track:
            if seq in sent.lower():
                sent_list_result.append(sent)
    return sent_list_result

In [24]:
%%time
df_sample['mention_ukr'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, seqs_to_track))

CPU times: user 8.81 s, sys: 4.98 s, total: 13.8 s
Wall time: 18.3 s


In [25]:
data = df_sample['mention_ukr'].explode().unique()

In [26]:
df_mentions = pd.DataFrame({'text': data[:]})

In [27]:
df_mentions.to_csv('ukr_mentions.csv', index = False, header=True)

We use Deep pavlov sentiment classifier for further filtering out neutral and positive stances.

In [28]:
import torch
from transformers import AutoModelForSequenceClassification
from transformers import BertTokenizerFast

In [29]:
tokenizer = BertTokenizerFast.from_pretrained('blanchefort/rubert-base-cased-sentiment')
model = AutoModelForSequenceClassification.from_pretrained('blanchefort/rubert-base-cased-sentiment', return_dict=True)


def predict(text):
    inputs = tokenizer(text, max_length=512, padding=True, truncation=True, return_tensors='pt')
    outputs = model(**inputs)
    predicted = torch.nn.functional.softmax(outputs.logits, dim=1)
    predicted = torch.argmax(predicted, dim=1).numpy()
    return predicted

We add sequences that correspond to the more radical wording

In [30]:
str_to_track = ['укры', 'хох', 'хохл', 'рейх', 'фаш']

In [31]:
%%time
df_sample['mention_ukr_radical'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, str_to_track))

CPU times: user 13.9 s, sys: 4.99 s, total: 18.9 s
Wall time: 21.3 s


In [32]:
data_radical = df_sample['mention_ukr_radical'].explode().unique()
df_radical = pd.DataFrame({'text': data_radical[:]})

In [33]:
%%time
df_radical = df_radical[1:]

CPU times: user 84 µs, sys: 32 µs, total: 116 µs
Wall time: 209 µs


In [34]:
%%time
df_radical['tone'] = df_radical['text'].apply(lambda x: predict(x))

CPU times: user 14min 58s, sys: 7min 18s, total: 22min 17s
Wall time: 15min 31s


In [35]:
df_radical = df_radical[df_radical['tone']==2]

In [ ]:
df_radical.to_csv('df_radical.csv', index = False, header=True)

We briefly experiment with other roots but don't rely on these results further.

In [37]:
strings = ["укр", "Укр" "хох", "Хох"]
df_filtered = df_sample[df_sample["message"].str.contains('|'.join(strings))]

In [38]:
len(df_sample)

1367309

In [39]:
df_tone_sample = df_filtered.sample(10000, random_state=42).copy()

In [40]:
%%time
df_tone_sample['tone'] = df_tone_sample['message'].apply(lambda x: predict(x))

CPU times: user 28min 22s, sys: 9min 10s, total: 37min 32s
Wall time: 17min 42s


In [41]:
df_tone_sample[df_tone_sample['tone']==2]

,index,id,date,message,type,channel_name,lang,tokenized_sents,random_sents,mention_ukr,mention_ukr_radical,tone
1269901,817024,166442.0,2022-11-15 22:48:27+00:00,Что известно о взрывах на востоке Польши ◾️Ве...,text,tass_agency,ru,[Что известно о взрывах на востоке Польши ◾️В...,Что известно о взрывах на востоке Польши ◾️Ве...,[],[],[2]
640138,1179620,1127.0,2021-04-15 15:08:05+00:00,"Российский военкор рассказал, когда ждать пров...",photo,swodki,ru,"[Российский военкор рассказал, когда ждать про...","Российский военкор рассказал, когда ждать пров...",[],[],[2]
225021,5409015,45661.0,2022-04-28 13:36:06+00:00,"♨️ Украинские Т-каналы пишут, что в Киеве и Од...",video,boris_rozhin,ru,"[♨️ Украинские Т-каналы пишут, что в Киеве и О...","♨️ Украинские Т-каналы пишут, что в Киеве и Од...",[],[],[2]
620894,1050852,197532.0,2022-12-01 21:05:20+00:00,Фронтовая сводка в направлении Сватово - Креме...,video,swodki,ru,[Фронтовая сводка в направлении Сватово - Крем...,Фронтовая сводка в направлении Сватово - Креме...,[],[Фронтовая сводка в направлении Сватово - Крем...,[2]
450236,1290156,47765.0,2022-11-27 16:00:01+00:00,Украинская энергосистема пожирает сама себя: ч...,photo,readovkanews,ru,[Украинская энергосистема пожирает сама себя: ...,Украинская энергосистема пожирает сама себя: ч...,[],[],[2]
...,...,...,...,...,...,...,...,...,...,...,...,...
303016,2794715,9437.0,2022-02-27 13:46:26+00:00,⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские во...,text,skabeeva,ru,[⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские в...,⚡️⚡️⚡️СРОЧНО МИНОБОРОНЫ РОССИИ Украинские во...,[],[],[2]
75558,3846260,157781.0,2022-03-27 15:49:05+00:00,⚡️Глава СК РФ Александр Бастрыкин поручил расс...,text,glavmedia,ru,[⚡️Глава СК РФ Александр Бастрыкин поручил рас...,⚡️Глава СК РФ Александр Бастрыкин поручил расс...,[],[],[2]
597630,1096190,130503.0,2022-07-09 07:10:02+00:00,Основные итоги СВО на Украине за 8 июля 2022 г...,video,swodki,ru,[Основные итоги СВО на Украине за 8 июля 2022 ...,Основные итоги СВО на Украине за 8 июля 2022 г...,[],[],[2]
733911,2235534,24801.0,2022-09-01 18:00:00+00:00,⏱ Главные события на 21:00 ▪️Делегация МАГАТЭ...,text,OstashkoNews,ru,[⏱ Главные события на 21:00 ▪️Делегация МАГАТ...,⏱ Главные события на 21:00 ▪️Делегация МАГАТЭ...,[],[],[2]


In [162]:
data2 = df_tone_sample['tokenized_sents'][df_tone_sample['tone']==2].explode().unique()
df_ukr_neg = pd.DataFrame({'text': data2[:]})
df_ukr_neg.to_csv('ukr_neg.csv', index = False, header=True)

In [149]:
strings2 = ["укры", "Укры" "хох", "Хох"]
df_filtered2 = df_sample[df_sample["message"].str.contains('|'.join(strings))]

In [150]:
len(df_filtered2)

94927

In [151]:
df_tone_sample2 = df_filtered2.sample(5000, random_state=42).copy()

In [152]:
df_tone_sample2['tone'] = df_tone_sample2['message'].apply(lambda x: predict(x))

In [160]:
data3 = df_tone_sample2['tokenized_sents'][df_tone_sample2['tone']==2].explode().unique()
df_ukr_neg2 = pd.DataFrame({'text': data3[:]})
df_ukr_neg2.to_csv('ukr_neg2.csv', index = False, header=True)

### For w2v training we take the 10% sample and preprocess it.

We train w2v for 10 and then for 20 epochs, and select the one with 20 ep as final.

In [ ]:
df_small_sample

In [ ]:
len(df_small_sample)

In [ ]:
splitters = ['\n', '\t', '\\n','\xa0', '\u200b']
df_small_sample['message_no_breaks'] = df_small_sample['message'].str.replace('\n|\t|\\n', ' ', case=False)

In [ ]:
def get_emoji_count(text):
    return collections.Counter([match["emoji"] for word in text for match in emoji.emoji_list(word)])

In [ ]:
def remove_emoji(string):
    emoji_pattern = re.compile("["
                           u"\U0001F600-\U0001F64F"  # emoticons
                           u"\U0001F300-\U0001F5FF"  # symbols & pictographs
                           u"\U0001F680-\U0001F6FF"  # transport & map symbols
                           u"\U0001F1E0-\U0001F1FF"  # flags (iOS)
                           u"\U00002702-\U000027B0"
                           u"\U000024C2-\U0001F251"
                           "]+", flags=re.UNICODE)
    return emoji_pattern.sub(r' ', string)

In [ ]:
%%time
df_small_sample['message_no_emoji'] = df_small_sample['message_no_breaks'].apply(lambda x: remove_emoji(x))

In [ ]:
df_small_sample['message_no_urls'] = df_small_sample['message_no_emoji'].str.replace('http\S+|www.\S+', '', case=False)

In [ ]:
def cut_mentions(list_of_mentions, text):
    for word in list_of_mentions:
        wrd = '@' + word
        text = text.replace(wrd, '')
    return text.strip()

In [ ]:
df_small_sample['mentions'] = df_small_sample['message_no_urls'].str.findall(r"@([a-zA-Z0-9_]{1,50})")

In [ ]:
%%time
df_small_sample['message_no_mentions'] = df_small_sample.apply(lambda x: cut_mentions(x['mentions'], x['message_no_urls']), axis=1)

In [ ]:
to_replace = ["«", "»", "“"]

df_small_sample['message_no_mentions'] = df_small_sample['message_no_mentions'].str.replace('|'.join(to_replace),'"')

In [ ]:
def cut_hasgtags(set_of_hashtags, text):
    for word in list(set_of_hashtags):
        text = text.replace(word, '')
    return text.strip()

In [ ]:
%%time
df_small_sample['hashtags'] = df_small_sample['message_no_mentions'].apply(lambda x: {word.strip(",").strip(".") for word in x.split() if word.startswith("#")})

In [ ]:
df_small_sample['message_no_hashtags'] = df_small_sample.apply(lambda x: cut_hasgtags(x['hashtags'], x['message_no_mentions']), axis=1)


In [ ]:
punctuation_minimal = '!(),-.:;?%'

cyrillic_letters = u"абвгдеёжзийклмнопрстуфхцчшщъыьэюяАБВГДЕЁЖЗИЙКЛМНОПРСТУФХЦЧШЩЪЫЬЭЮЯ"
latin_letters_numbers = "1234567890abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ "
allowed_symbols = cyrillic_letters+latin_letters_numbers + punctuation_minimal

In [ ]:
def clean_text(string, allowed_symbols=allowed_symbols):
    getVals = list(filter(lambda x: x in allowed_symbols, string))
    result = "".join(getVals)
    result = re.sub(' +', ' ', result)

    return result

In [ ]:
%%time
df_small_sample['message_clean'] = df_small_sample['message_no_hashtags'].apply(lambda x: clean_text(x))

In [ ]:
only_letters = cyrillic_letters+latin_letters_numbers

In [ ]:
%%time
df_small_sample['message_words_only_lower'] = df_small_sample['message_clean'].apply(lambda x: clean_text(x, allowed_symbols=only_letters).lower())

In [ ]:
with pd.option_context('display.max_colwidth', None):
    print(df_small_sample['message_words_only_lower'].sample(1))

In [ ]:
df_small_sample[['message_words_only_lower']].to_csv('small_sample.csv', header=True)

In [3]:
import spacy

In [4]:
nlp = spacy.load('ru_core_news_md')

In [5]:
def tokenize_spacy(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append([token.text, token.pos_, token.dep_, token.lemma_, token.morph])
    return result

In [ ]:
%%time
df_small_sample['tokens'] = df_small_sample['message_words_only_lower'].apply(lambda x: tokenize_spacy(x))

In [ ]:
df_small_sample['tokens']

In [244]:
df_small_sample.to_csv('df_small_sample.csv')

In [1]:
import pandas as pd

In [2]:
df_small_sample = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/data/df_small_sample.csv')

In [13]:
def get_lemma(list_of_lists):
    result = []
    for list in list_of_lists:
        result.append(list[3])
    return result

In [6]:
def tokenize_spacy_lemma(text):
    doc = nlp(text)
    result = []
    for token in doc:
        result.append(token.lemma_)
    return result

In [15]:
%%time
# test_set = df_small_sample['tokens'].sample(1000).apply(lambda x: get_lemma(x))

CPU times: user 1e+03 ns, sys: 0 ns, total: 1e+03 ns
Wall time: 18.8 µs


In [16]:
# df_small_sample['tokens'][0][0][3]

In [ ]:
%%time
df_small_sample['lemmas'] = df_small_sample['tokens'].apply(lambda x: get_lemma(x))

In [32]:
df_small_sample['tokens']

0         [['дмитрий', 'PROPN', 'nsubj', 'дмитрий', Anim...
1         [['мид', 'NOUN', 'nsubj', 'мид', Animacy=Inan|...
2         [['санкции', 'NOUN', 'nsubj', 'санкция', Anima...
3         [['новая', 'ADJ', 'amod', 'новый', Case=Nom|De...
4         [['эксклюзив', 'NOUN', 'nsubj', 'эксклюзив', A...
                                ...                        
683649    [['франция', 'PROPN', 'nsubj', 'франция', Anim...
683650    [['возможной', 'ADJ', 'amod', 'возможный', Cas...
683651    [['сегодня', 'ADV', 'advmod', 'сегодня', Degre...
683652    [['стрелка', 'NOUN', 'obj', 'стрелок', Animacy...
683653    [['мошенники', 'NOUN', 'nsubj', 'мошенник', An...
Name: tokens, Length: 683654, dtype: object

In [ ]:
documents = df_small_sample['lemmas'].tolist()

In [ ]:
documents

In [270]:
import gensim

In [273]:
%%time
model = gensim.models.Word2Vec (documents, vector_size=150, window=10, min_count=2, workers=10)
model.train(documents,total_examples=len(documents),epochs=10)

CPU times: user 21min 19s, sys: 1min 55s, total: 23min 14s
Wall time: 6min 48s


(333227208, 381780410)

In [282]:
w1 = "украинец" #ы
model.wv.most_similar (positive=w1)

[('поляк', 0.7101138234138489),
 ('русских', 0.7060922980308533),
 ('русские', 0.683661699295044),
 ('европеец', 0.6558117866516113),
 ('немец', 0.6476935744285583),
 ('американец', 0.6276693344116211),
 ('бандеровец', 0.6234453320503235),
 ('британец', 0.6072903275489807),
 ('украина', 0.6048406362533569),
 ('киевлянин', 0.6004078388214111)]

In [295]:
w2 = "укронацисты" #ы
model.wv.most_similar (positive=w2)

[('всушники', 0.7371035814285278),
 ('всу', 0.7126280069351196),
 ('укры', 0.6975746154785156),
 ('укроп', 0.6604259014129639),
 ('украинские', 0.6520400643348694),
 ('укровояки', 0.6500487923622131),
 ('нацбаты', 0.6468362808227539),
 ('укронацистов', 0.6359423995018005),
 ('нацики', 0.624232292175293),
 ('всуки', 0.6043425798416138)]

In [284]:
model.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_10ep_10w.model")

In [285]:
%%time
w2v_small_sample_20ep_20w = gensim.models.Word2Vec (documents, vector_size=150, window=20, min_count=2, workers=10)
w2v_small_sample_20ep_20w.train(documents,total_examples=len(documents),epochs=20)

CPU times: user 43min 46s, sys: 3min 24s, total: 47min 11s
Wall time: 13min 40s


(666454099, 763560820)

In [306]:
w2v_small_sample_20ep_20w.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_20ep_20w.model")

In [296]:
w1 = "украинец" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w1)

[('русских', 0.6968995332717896),
 ('русские', 0.6964977383613586),
 ('украина', 0.651254415512085),
 ('поляк', 0.6197736859321594),
 ('киев', 0.6087530851364136),
 ('европеец', 0.5969522595405579),
 ('русским', 0.5910526514053345),
 ('русскими', 0.585874080657959),
 ('бандеровец', 0.5767456293106079),
 ('нацист', 0.5747347474098206)]

In [303]:
w2 = "укр" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w2)

[('хлопцы', 0.525049090385437),
 ('укрорейха', 0.49813905358314514),
 ('хохол', 0.4795973598957062),
 ('украинский', 0.4786289632320404),
 ('укро', 0.47841185331344604),
 ('укроп', 0.475413054227829),
 ('уничтоженный', 0.4753034710884094),
 ('украинским', 0.47485196590423584),
 ('артой', 0.46757906675338745),
 ('всушник', 0.46454688906669617)]

In [298]:
w3 = "укр" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хлопцы', 0.525049090385437),
 ('укрорейха', 0.49813905358314514),
 ('хохол', 0.4795973598957062),
 ('украинский', 0.4786289632320404),
 ('укро', 0.47841185331344604),
 ('укроп', 0.475413054227829),
 ('уничтоженный', 0.4753034710884094),
 ('украинским', 0.47485196590423584),
 ('артой', 0.46757906675338745),
 ('всушник', 0.46454688906669617)]

In [301]:
w3 = "хлопцы" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хлопец', 0.6090108156204224),
 ('драпать', 0.6059606671333313),
 ('хохлы', 0.5939660668373108),
 ('нацики', 0.5922543406486511),
 ('всушников', 0.5839340686798096),
 ('всушники', 0.578698456287384),
 ('хохол', 0.5765518546104431),
 ('всушник', 0.5621557831764221),
 ('нахй', 0.5591046810150146),
 ('пацаны', 0.5548537969589233)]

In [302]:
w3 = "хохлы" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w3)

[('хохол', 0.6874178051948547),
 ('москали', 0.6082358360290527),
 ('хлопцы', 0.5939660668373108),
 ('блть', 0.592091977596283),
 ('укроп', 0.5853246450424194),
 ('пацаны', 0.5584353804588318),
 ('всушников', 0.541182279586792),
 ('шо', 0.5330790281295776),
 ('свидомые', 0.5313231348991394),
 ('москаль', 0.5298460125923157)]

In [299]:
w4 = "бандеровец" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w4)

[('нацист', 0.7867278456687927),
 ('нациков', 0.6760679483413696),
 ('фашист', 0.6734396815299988),
 ('нацики', 0.6429235935211182),
 ('неонацист', 0.63155597448349),
 ('оккупант', 0.6156263947486877),
 ('бандеры', 0.6109479069709778),
 ('недобиток', 0.5980472564697266),
 ('бандеровскую', 0.5909456610679626),
 ('украинец', 0.5767455697059631)]

In [300]:
w5 = "бандеры" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w5)

[('шухевича', 0.7994802594184875),
 ('бандеру', 0.6805114150047302),
 ('упа', 0.675289511680603),
 ('нацист', 0.6455380916595459),
 ('нацистский', 0.6343949437141418),
 ('бандера', 0.6273082494735718),
 ('бандеровец', 0.6109479069709778),
 ('фашист', 0.5912745594978333),
 ('коллаборационист', 0.5839884877204895),
 ('нацисткой', 0.5792540311813354)]

In [307]:
%%time
w2v_small_sample_30ep_20w = gensim.models.Word2Vec (documents, vector_size=150, window=20, min_count=2, workers=10)
w2v_small_sample_30ep_20w.train(documents,total_examples=len(documents),epochs=30)

CPU times: user 1h 33s, sys: 4min 53s, total: 1h 5min 26s
Wall time: 22min 44s


(999663280, 1145341230)

In [309]:
w2v_small_sample_30ep_20w.save("/Users/katerynaburovova/PycharmProjects/dehumanization/w2v_models/w2v_small_sample_30ep_20w.model")

### Compiling the list of group labels for pre-selection

In [304]:
w = "укры" #ы
w2v_small_sample_20ep_20w.wv.most_similar (positive=w)

[('укров', 0.6507847309112549),
 ('укронацисты', 0.6384262442588806),
 ('хохол', 0.6045107245445251),
 ('всу', 0.5642962455749512),
 ('укроп', 0.5553898215293884),
 ('всушники', 0.5513160824775696),
 ('арта', 0.5499454140663147),
 ('укропов', 0.5253047347068787),
 ('укропские', 0.5176306962966919),
 ('грады', 0.5170502066612244)]

In [382]:
w = "укропитеки" #ы
model.wv.most_similar (positive=w, topn=20)

[('побратим', 0.5724674463272095),
 ('хохол', 0.5294627547264099),
 ('укры', 0.5038765072822571),
 ('хаймарсы', 0.48131513595581055),
 ('машенька', 0.4766599237918854),
 ('м14', 0.4630703032016754),
 ('янки', 0.46303218603134155),
 ('нацики', 0.4626290500164032),
 ('рагулье', 0.46123623847961426),
 ('русские', 0.4609838128089905),
 ('рагулям', 0.4592565596103668),
 ('укроп', 0.4564368724822998),
 ('садыки', 0.4542776644229889),
 ('всушники', 0.4501821994781494),
 ('нихя', 0.44559958577156067),
 ('хохлы', 0.4436476528644562),
 ('всушниками', 0.4421617090702057),
 ('вояка', 0.4414631426334381),
 ('укропропаганда', 0.4394609332084656),
 ('соляру', 0.43590500950813293)]

In [378]:
w = "свинособак" #ы
w2v_small_sample_30ep_20w.wv.most_similar (positive=w, topn=20)

[('москаль', 0.5343905687332153),
 ('нацист', 0.5062564015388489),
 ('бандеровец', 0.477138489484787),
 ('нациков', 0.47089558839797974),
 ('свинособаки', 0.47018522024154663),
 ('нацики', 0.46963152289390564),
 ('мразей', 0.46705949306488037),
 ('полицай', 0.4554937779903412),
 ('справжних', 0.44639554619789124),
 ('азiва', 0.4461626410484314),
 ('укров', 0.44432422518730164),
 ('хохол', 0.44374388456344604),
 ('побратим', 0.4429384171962738),
 ('сепаров', 0.4396215081214905),
 ('фашист', 0.4381740093231201),
 ('хихиканье', 0.4364808201789856),
 ('укрорейха', 0.4360989034175873),
 ('наци', 0.43421411514282227),
 ('каратель', 0.4289534091949463),
 ('тих', 0.4284592270851135)]

In [329]:
list_of_labels = ['бандеры',
                  'бандеровец',
                  'неонацист',
                  'неонацисты',
                  'хохол',
                  'хохлы',
                  'хлопцы',
                  'укроп',
                  'всушники',
                  'всушник',
                  'свидомые',
                  'укрорейх',
                  'укроп',
                  'укропы',
                  'всуки',
                  'укронацисты',
                  'укронацист',
                  'укронацики',
                  'укронацик',
                  'укро-', #складені імена через дефіс, наскільки я розумію
                  'укровояки',
                  'майданутые',
                  'салорейх',
                  'хрюн',
                  'чуб',
                  ]

In [330]:
list_of_roots = ['бандер',
                  'неонац',
                  'хохол',
                  'хохл',
                  'хлопц',
                  'укроп',
                  'всушник',
                  'свидом',
                  'орейх',
                  'всук',
                  'укронац',
                  'укро-', #складені імена через дефіс, наскільки я розумію
                    'укро ',
                  'укровояк',
                  'майданут',
                  'хрюн',
                  'чуб',
                  ]

In [336]:
%%time
df_sample['mentions_roots'] = df_sample['tokenized_sents'].apply(lambda x: find_sentences_w_sec(x, list_of_roots))

CPU times: user 41.2 s, sys: 4.06 s, total: 45.3 s
Wall time: 54.6 s


In [338]:
data = df_sample['mentions_roots'].explode().unique()
df_w2v_roots = pd.DataFrame({'text': data[:]})
df_w2v_roots.to_csv('df_w2v_roots.csv')

We then manually process the df_w2v_roots to select all sentences that have the most risk to contain dehumanization signals and join the prevous selections. The final dataset is processed to be written as separate rows and imported to Labelbox.

In [2]:
import pandas as pd
df_annotation = pd.read_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_v_1.csv', header=0)

In [348]:
# writePath = '/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/data/ann_dataset.txt'
# with open(writePath, 'a') as f:
#     dfAsString = df_annotation.to_string(header=False, index=False)
#     f.write(dfAsString)

In [388]:
df_annotation = df_annotation.sample(frac=1).reset_index(drop=True)

In [ ]:
df_annotation.to_csv('/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/data/df_annotation.csv')

In [390]:
for i, row in df_annotation.iterrows():
    # Create a filename for each row
    filename = f"/Users/katerynaburovova/PycharmProjects/dehumanization/annotation/dataset_shuffled/row_{i}.txt"

    # Write the contents of each row to a separate text file
    with open(filename, "w", encoding='utf-8') as f:
        f.write(str(row.to_dict()['text']))

In [ ]:
len(df_small_sample) / len(df_posts)